In [1]:
from __future__ import print_function
import keras
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
batch_size = 32 
num_classes = 10
epochs = 100
data_augmentation = True

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
model = Sequential()

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(80, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.25))


model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

<IPython.core.display.Javascript object>

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 48)        13872     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 48)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 48)        0         
                                                      

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [6]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test), callbacks=[tbCallBack])

Using real-time data augmentation.
Epoch 1/100
1562/1562 [==============================] - 136s 86ms/step - loss: 2.0160 - accuracy: 0.2399 - val_loss: 1.7598 - val_accuracy: 0.3605
Epoch 2/100
1562/1562 [==============================] - 134s 85ms/step - loss: 1.7496 - accuracy: 0.3471 - val_loss: 1.5763 - val_accuracy: 0.4306
Epoch 3/100
1562/1562 [==============================] - 137s 88ms/step - loss: 1.6374 - accuracy: 0.3930 - val_loss: 1.4687 - val_accuracy: 0.4739
Epoch 4/100
1562/1562 [==============================] - 137s 87ms/step - loss: 1.5591 - accuracy: 0.4284 - val_loss: 1.3961 - val_accuracy: 0.5003
Epoch 5/100
1562/1562 [==============================] - 136s 87ms/step - loss: 1.4985 - accuracy: 0.4551 - val_loss: 1.3387 - val_accuracy: 0.5173
Epoch 6/100
1562/1562 [==============================] - 138s 88ms/step - loss: 1.4423 - accuracy: 0.4762 - val_loss: 1.3774 - val_accuracy: 0.4981
Epoch 7/100
1562/1562 [==============================] - 140s 90ms/step - los

Epoch 56/100
1562/1562 [==============================] - 110s 71ms/step - loss: 0.7590 - accuracy: 0.7359 - val_loss: 0.6520 - val_accuracy: 0.7716
Epoch 57/100
1562/1562 [==============================] - 111s 71ms/step - loss: 0.7543 - accuracy: 0.7353 - val_loss: 0.6570 - val_accuracy: 0.7738
Epoch 58/100
1562/1562 [==============================] - 112s 72ms/step - loss: 0.7516 - accuracy: 0.7364 - val_loss: 0.6707 - val_accuracy: 0.7673
Epoch 59/100
1562/1562 [==============================] - 109s 70ms/step - loss: 0.7460 - accuracy: 0.7389 - val_loss: 0.6482 - val_accuracy: 0.7744
Epoch 60/100
1562/1562 [==============================] - 107s 68ms/step - loss: 0.7409 - accuracy: 0.7398 - val_loss: 0.6918 - val_accuracy: 0.7626
Epoch 61/100
1562/1562 [==============================] - 110s 71ms/step - loss: 0.7360 - accuracy: 0.7422 - val_loss: 0.6457 - val_accuracy: 0.7739
Epoch 62/100
1562/1562 [==============================] - 118s 76ms/step - loss: 0.7341 - accuracy: 0.7444

In [7]:
scores=model.evaluate(x_test,y_test,verbose=1)
print("Test Loss: ",scores[0])
print("Test Accuracy: ",scores[1])

313/313 [==============================] - 5s 15ms/step - loss: 0.5722 - accuracy: 0.8041
Test Loss:  0.5721825957298279
Test Accuracy:  0.804099977016449
